In [36]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import scipy
from datetime import datetime
import matplotlib.pyplot as plt

In [54]:
filepath = "Data/Belgium/ELIA/ods001.csv"

In [55]:
data = pd.read_csv(filepath, delimiter=";")[749:][::-1].reset_index().drop("index", axis=1)

In [39]:
data["Datetime"] = pd.to_datetime(data["Datetime"], errors='coerce', utc=True)
data["Datetime"] = data["Datetime"].dt.tz_convert("UTC")

In [56]:
data.head()

,Datetime,Resolution code,Total Load,Most recent forecast,Most recent P10,Most recent P90,Day-ahead 6PM forecast,Day-ahead 6PM P10,Day-ahead 6PM P90,Week-ahead forecast
0,2015-01-01T00:00:00+01:00,PT15M,10142.19,9496.05,8871.71,10120.38,9697.59,9076.64,10318.53,9833.37
1,2015-01-01T00:15:00+01:00,PT15M,10051.28,9329.17,8715.80,9942.53,9526.54,8916.55,10136.53,9653.31
2,2015-01-01T00:30:00+01:00,PT15M,9952.87,9174.72,8571.51,9777.92,9367.66,8767.84,9967.47,9473.22
3,2015-01-01T00:45:00+01:00,PT15M,9821.78,9025.46,8432.07,9618.85,9216.09,8625.97,9806.20,9319.45
4,2015-01-01T01:00:00+01:00,PT15M,9755.00,9222.33,8880.23,9564.44,10130.36,9777.87,10482.86,9203.25


In [40]:
data['Total Load'] = data['Total Load'].interpolate(method='linear')

In [41]:
data["Year"] = [i.year for i in data["Datetime"]]
data["Month"] = [i.month for i in data["Datetime"]]
data["Day"] = [i.day for i in data["Datetime"]]
data["Hour"] = [i.hour for i in data["Datetime"]]
data["Minute"] = [i.minute for i in data["Datetime"]]
data["Season"] = data["Month"]//4

Lockdown dates:
- 18/03/2020 -> 04/05/2020
- 30/10/2020 -> 15/12/2020
- 20/11/2020 -> 20/01/2022

In [42]:
data["Lockdown"] = 0

In [43]:
data.loc[182784:187296, "Lockdown"] = 1
data.loc[204388:208804, "Lockdown"] = 1
data.loc[212260:247300, "Lockdown"] = 1

In [44]:
data["Difference with previous load"] = [0] + list(np.array(data["Total Load"][1:].to_list()) - np.array(data["Total Load"][:-1].to_list()))

In [45]:
data.head()

,Datetime,Resolution code,Total Load,Most recent forecast,Most recent P10,Most recent P90,Day-ahead 6PM forecast,Day-ahead 6PM P10,Day-ahead 6PM P90,Week-ahead forecast,Year,Month,Day,Hour,Minute,Season,Lockdown,Difference with previous load
0,2014-12-31 23:00:00+00:00,PT15M,10142.19,9496.05,8871.71,10120.38,9697.59,9076.64,10318.53,9833.37,2014,12,31,23,0,3,0,0.00
1,2014-12-31 23:15:00+00:00,PT15M,10051.28,9329.17,8715.80,9942.53,9526.54,8916.55,10136.53,9653.31,2014,12,31,23,15,3,0,-90.91
2,2014-12-31 23:30:00+00:00,PT15M,9952.87,9174.72,8571.51,9777.92,9367.66,8767.84,9967.47,9473.22,2014,12,31,23,30,3,0,-98.41
3,2014-12-31 23:45:00+00:00,PT15M,9821.78,9025.46,8432.07,9618.85,9216.09,8625.97,9806.20,9319.45,2014,12,31,23,45,3,0,-131.09
4,2015-01-01 00:00:00+00:00,PT15M,9755.00,9222.33,8880.23,9564.44,10130.36,9777.87,10482.86,9203.25,2015,1,1,0,0,0,0,-66.78


In [46]:
data.to_csv("Processed_data.csv")

Seasonality Analysis

In [ ]:
data = pd.read_csv("Processed_data.csv", sep=",")
data.drop(columns=['Unnamed: 0'], inplace = True)

data['Datetime'] = pd.to_datetime(data['Datetime'], format="%Y-%m-%d %H:%M:%S")
data = data.set_index('Datetime')

In [ ]:
def plot_seasonal_patterns(data, index, columns, values, title, xlabel, ylabel, legend_title, figsize=(12, 6)):
    '''
    Function to plot seasonal patterns.
    
    :param data: DataFrame containing the data.
    :param index: Column(s) to use as index for pivoting (e.g., 'Month', 'Day', 'Hour').
    :param columns: Column(s) to use for the legend (e.g., 'Year', ['Month', 'Year']).
    :param values: Column to use for the values (e.g., 'Total Load').
    :param title: Title of the plot.
    :param xlabel: Label for the x-axis.
    :param ylabel: Label for the y-axis.
    :param legend_title: Title of the legend.
    :param figsize: Size of the figure (default is (12, 6)).
    '''
    pivot_table = data.pivot_table(index=index, columns=columns, values=values)
    
    plt.figure(figsize=figsize)
    plt.plot(pivot_table)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    #plt.legend(pivot_table.columns, title=legend_title, bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()


In [ ]:
data['Total Load'].plot(ylabel='Total Load')

In [ ]:
# Seasonal patterns by month of the year
plot_seasonal_patterns(
    data=data,
    index='Month',
    columns='Year',
    values='Total Load',
    title="Seasonal Patterns by Month",
    xlabel="Month",
    ylabel="Average Total Load",
    legend_title="Year"
)

# Seasonal patterns by day of the month
plot_seasonal_patterns(
    data=data,
    index='Day',
    columns=['Month', 'Year'],
    values='Total Load',
    title="Seasonal Patterns by Day of the Month",
    xlabel="Day of the Month",
    ylabel="Average Total Load",
    legend_title="Month/Year"
)

# Seasonal patterns by hour of the day
plot_seasonal_patterns(
    data=data,
    index='Hour',
    columns=['Day', 'Month', 'Year'],
    values='Total Load',
    title="Seasonal Patterns by Hour of the Day",
    xlabel="Hour of the Day",
    ylabel="Average Total Load",
    legend_title="Day/Month/Year"
)

- The seasonality occurring every year is clearly visible.
- Seasonality is also observable on a monthly and hourly basis.
- Daily seasonality, however, is not observable.